# Coursera Capstone Final Project

I want to open a new Restaurant in Porto District with the district code in Portugal of 4. So I want to decide which is the best municipality place to open the new restaurant. My criteria is based on the amount of restaurants in each municipality vs. population density in each municipality.

In [80]:
#import the packages
import numpy as np
import pandas as pd
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from tqdm import tqdm
import requests
from collections import deque
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib import pyplot as plt

%matplotlib inline

In [81]:
#creates a source for the Beautiful Soup
df = pd.read_excel(r'pordata.xlsx') # Taken from www.pordata.pt
df = df[['Region','Municipality','Density']]
dfporto = df[df['Region'] == 'Porto']
dfporto.head(10)

,Region,Municipality,Density
24,Porto,Arouca,63.7
25,Porto,Espinho,1401.2
26,Porto,Gondomar,1255.5
27,Porto,Maia,1654.8
28,Porto,Matosinhos,2788.6
29,Porto,Oliveira de Azeméis,410.8
30,Porto,Paredes,549.2
31,Porto,Porto,5189.2
32,Porto,Póvoa de Varzim,759.6
33,Porto,Santa Maria da Feira,641.9


In [82]:
# All districts and municipalities of Portugal
code_data = pd.read_csv("concelhos.csv")[["nome_concelho","cod_distrito","cod_concelho"]] # taken from http://centraldedados.pt/codigos_postais/
code_data.columns = ['Municipality', 'DCode', 'MCode']
code_data.head(10)

,Municipality,DCode,MCode
0,Castelo de Paiva,1,6
1,Espinho,1,7
2,Estarreja,1,8
3,Santa Maria da Feira,1,9
4,Ílhavo,1,10
5,Mealhada,1,11
6,Murtosa,1,12
7,Oliveira de Azeméis,1,13
8,Oliveira do Bairro,1,14
9,Ovar,1,15


In [83]:
# Porto district code is 13
porto_data = code_data[code_data['DCode'] == 13]
porto_data.head(10)

,Municipality,DCode,MCode
178,Maia,13,6
179,Marco de Canaveses,13,7
180,Matosinhos,13,8
181,Paços de Ferreira,13,9
182,Paredes,13,10
183,Penafiel,13,11
184,Porto,13,12
185,Póvoa de Varzim,13,13
186,Santo Tirso,13,14
187,Valongo,13,15


In [84]:
# Copy MCode to df
dfporto = dfporto.merge(porto_data[["Municipality", "MCode"]])

dfporto

,Region,Municipality,Density,MCode
0,Porto,Gondomar,1255.5,4
1,Porto,Maia,1654.8,6
2,Porto,Matosinhos,2788.6,8
3,Porto,Paredes,549.2,10
4,Porto,Porto,5189.2,12
5,Porto,Póvoa de Varzim,759.6,13
6,Porto,Santo Tirso,500.7,14
7,Porto,Trofa,531.6,18
8,Porto,Valongo,1281.1,15
9,Porto,Vila do Conde,533.4,16


In [85]:
# Read latitude and longitude
coord_data = pd.read_csv("lat_long_f.csv", sep = ';', decimal=',')[['dd_desig', 'LOCALIDADE', 'CP4', 'LATITUDE', 'LONGITUDE']] # taken from http://centraldedados.pt/codigos_postais/
coord_data.columns = ['Region', 'Municipality', 'CPCode', 'Latitude', 'Longitude']
coord_porto = coord_data[coord_data['Region'] == 'Porto']
coord_porto.head()

,Region,Municipality,CPCode,Latitude,Longitude
183068,Porto,Cruzeiro,4615,41.322937,-8.144780
183069,Porto,Carvalheiras,4615,41.153729,-8.606222
183070,Porto,Campo,4615,41.322937,-8.144780
183071,Porto,Aboim,4600,41.330636,-8.071072
183072,Porto,Amarante,4600,41.262439,-8.082573


In [86]:
# Postal Code, Latitude and Longitude 
dfporto = dfporto.merge(coord_porto[["Municipality", 'Latitude', 'Longitude']])
df_porto = dfporto.groupby('Municipality').mean().reset_index()
df_porto

,Municipality,MCode,Latitude,Longitude
0,Gondomar,4,41.139991,-8.532800
1,Maia,6,41.232430,-8.606012
2,Matosinhos,8,41.188501,-8.689159
3,Paredes,10,41.198988,-8.333568
4,Porto,12,41.160364,-8.619520
5,Santo Tirso,14,41.342284,-8.484229
6,Trofa,18,41.379333,-8.209239
7,Valongo,15,41.194495,-8.503505
8,Vila Nova de Gaia,17,41.113921,-8.619381
9,Vila do Conde,16,41.358067,-8.744970


In [87]:
# Folium Print
address = 'Porto, Portugal'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Porto are {}, {}.'.format(latitude, longitude))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Porto are 41.1494512, -8.6107884.


In [88]:
locations = df_porto[['Latitude', 'Longitude']]
locationlist = locations.values.tolist()

map_porto = folium.Map(location=[latitude, longitude], zoom_start=10)

for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=df_porto['Municipality'][point]).add_to(map_porto)

map_porto

In [89]:
# Select first building (which in our case is BedokVille):
neighborhood_name = df_porto.loc[0, 'Municipality']
neighborhood_latitude = df_porto.loc[0, 'Latitude'] 
neighborhood_longitude = df_porto.loc[0, 'Longitude'] 
# limit of number of venues returned by Foursquare API
LIMIT = 100 
radius = 500

In [90]:
CLIENT_ID = 'ZC4GEQ0R3WUBDVREK3PBHNT0O3HVJFKF1TFYQPIQEINCPREG'
CLIENT_SECRET = 'EWPJRXRGGNTDSJKXCLT2XAA4SLIJLC0IERAF4YHNM3MKFYRC'
VERSION = '20180604'
        
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
# filter the category for each row
#nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head(20)

,name,categories,lat,lng
0,BeerDrive Caffe,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",41.140675,-8.530712
1,Confeitaria Avenida,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",41.138011,-8.532795
2,O Monte,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",41.142131,-8.536163
3,Basho Sushi House,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",41.143862,-8.535312
4,Monte Crasto,"[{'id': '4bf58dd8d48988d165941735', 'name': 'S...",41.142088,-8.536096
5,Tip-Off,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",41.140522,-8.531692
6,Madureiras Souto,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",41.135916,-8.530946
7,Restaurante O Brasinhas,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",41.137065,-8.531881
8,Cantinho Avenida,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",41.136619,-8.533706
9,Marpão,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",41.136524,-8.531875
